Hash Attack lab, implementing SHA-1 and and experminetally validating the difficulty of hash attacks

In [101]:
def Ch(x,y,z):
    x_and_y = x & y
    notx_and_z = ~x & z
    xored = x_and_y ^ notx_and_z
    return xored

In [102]:
def Parity(x,y,z):
    xored = x ^ y ^ z
    return xored

In [103]:
def Maj(x,y,z):
    x_and_y = x & y
    x_and_z = x & z
    y_and_z = y & z
    xored = x_and_y ^ x_and_z ^ y_and_z
    return xored

Function that calls the coresponding the above functions depedning on 

In [104]:
def ft(x,y,z,t):
    if t >= 0 and t <= 19:
        return Ch(x,y,z)
    if t >= 20 and t <= 39:
        return Parity(x,y,z)
    if t >= 40 and t <= 59:
        return Maj(x,y,z)
    if t >= 60 and t <= 79:
        return Parity(x,y,z)

In [105]:
def ROTL(n,x):
    a = x << n
    b = x>>(32-n)
    return (a|b)&0xffffffff

In [106]:
# SHA-1 Constants Kt
#        0-19       20-39      40-59      60-79   
Kt = [0x5a827999,0x6ed9eba1,0x8f1bbcdc,0xca62c1d6]
# do val//20 to get indexs if val = 29//20 = 1 

Preprocessing consists of three steps: padding the message,parsing the message
into message blocks,and setting the initial hash value

Padding the message, this is used to ensure the message is a multiple of 512 bits, 

In [107]:
# def Pad(word):
#     msg_size = len(word)*8
#     k = msg_size%448
#     padded_msg = ""
    
#     if msg_size >= 448:
#         k-= 64

#     for block in word:
#         for i in range(len(block)): 
#             #adding the binary value of word to the padded msg 
#             padded_msg += ''.join(f"{ord(block[i]):08b}")
            

#     padded_msg += "1" 
#     for i in range(448-k-1):
#         padded_msg += "0" 
#     #gettin the message size in binarry and adding zero's infront of it 
#     size = bin(msg_size) 
#     size = size[2:] 
#     padded_msg +=size.zfill(64) 

#     return padded_msg

def Pad(word):
    msg_size = len(word)*8

    #l + 1 + k = 448%512
    k = 448%512 - (msg_size%512) - 1    
    if k < 0:
        k = -(k)

    padded_msg = ""
    for block in word:
        for i in range(len(block)): 
            #adding the binary value of word to the padded msg 
            padded_msg += ''.join(f"{ord(block[i]):08b}")
    padded_msg += "1" 

    for i in range(k):
        padded_msg += "0" 
    
    #gettin the message size in binarry and adding zero's infront of it 
    size = bin(msg_size) 
    size = size[2:] 
    padded_msg +=size.zfill(64) 
    
    return padded_msg

Parsing the Message and its padding must be parsed into N m-bit blocks. 16 M blocks with 32 bit words

In [108]:
#16 blocks of 32 bit words 
def Parsing(word):
    M = []
    for i in range(16):
        block = " "
        for j in range(32):
            block+=(word[(i*32)+j])
        M.append(int(block,2))
    return M

Before hash computation begins for each of the secure hash algorithms, the initial hash value,H0 must be set. The size and number of words in H0 depends on the message digest size.

In [109]:
def H0():
    H = [0x67452301, 0xefcdab89, 0x98badcfe, 0x10325476, 0xc3d2e1f0]
    return H

SHA-1 Hash Computation, we will use the constatns and functions used above to now implement Hash Computation

In [110]:
#preparing the message schedule Wt
def Schedule_WT(M):
    W = [0] * 80
    for t in range(0,80): 
        if t < 16: 
            #break Mi ininto 16 words W0, W1 .. W15 
            W[t] = M[t]   
        if (t >= 16): 
            w = W[t-3] ^ W[t-8] ^ W[t-14] ^ W[t-16]
            W[t] = ROTL(1,w)
    return W

In [111]:
# #SHA 1 has a has value of five 32-bit words
def SHA1(word = "abc"):
    H = H0()
    padded_msg = Pad(word)
    N = len(padded_msg)//512 
    for i in range(1,N+1):
        #get the corressponding block
        M = Parsing(padded_msg[(i-1)*512:i*512])

        Wt = Schedule_WT(M)


        
        a = H[0]
        b = H[1]
        c = H[2]
        d = H[3]
        e = H[4]
            
        for t in range(80):
            T = (ROTL(5,a) + ft(b,c,d,t) + e + Kt[t//20] + Wt[t]) & 0xffffffff
            e = d
            d = c
            c = ROTL(30,b)
            b = a
            a = T


        

        H[0] = (H[0] + a)&0xffffffff
        H[1] = (H[1] + b)&0xffffffff
        H[2] = (H[2] + c)&0xffffffff
        H[3] = (H[3] + d)&0xffffffff
        H[4] = (H[4] + e)&0xffffffff

    string = f'{hex(H[0])[2:]}{hex(H[1])[2:]}{hex(H[2])[2:]}{hex(H[3])[2:]}{hex(H[4])[2:]}'
    return string

In [112]:
test1 = "This is a test of SHA-1."
test2 = "Kerckhoff's principle is the foundation on which modern cryptography is built."
test3 = "SHA-1 is no longer considered a secure hashing algorithm."
test4 = "SHA-2 or SHA-3 should be used in place of SHA-1."
test5 = "Never roll your own crypto!"

print(SHA1(test1))
print(SHA1(test2))
print(SHA1(test3))
print(SHA1(test4))
print(SHA1(test5))

8e35b47213acb9fa620d8e884d3f6338166f34d7
f801ea3e4c55ca85928bbf1bb24776d61e3fe09
179f3c79f6bdf85e49666b3ea0f1308959e89208
dd102182aabb5778e925eb2f536bab904b97c9b5
ae912752721c0f7b5857cc8c314fb9a3e94ca1c0


In [113]:
def SHA1_MOD(mac, m):
    H = mac

    m1_hash_end = int(len(mac)/5)
    # m1_hash = m1_hash[m1_hash_end:]
    H = [
    int(mac[m1_hash_end*0:m1_hash_end*1],16),
    int(mac[m1_hash_end*1:m1_hash_end*2],16),
    int(mac[m1_hash_end*2:m1_hash_end*3],16), 
    int(mac[m1_hash_end*3:m1_hash_end*4],16), 
    int(mac[m1_hash_end*4:m1_hash_end*5],16),
    ]
    #print(hex(H[0]),hex(H[1]),hex(H[2]),hex(H[3]))
    padded_msg = Pad(m)
    #print(padded_msg)
    #print(len(padded_msg))
    N = len(padded_msg)//512 
    for i in range(N,N+1):
        #get the corressponding block
        M = Parsing(padded_msg[(i-1)*512:i*512])

        Wt = Schedule_WT(M)


        
        a = H[0]
        b = H[1]
        c = H[2]
        d = H[3]
        e = H[4]
            
        for t in range(80):
            T = (ROTL(5,a) + ft(b,c,d,t) + e + Kt[t//20] + Wt[t]) & 0xffffffff
            e = d
            d = c
            c = ROTL(30,b)
            b = a
            a = T


        

        H[0] = (H[0] + a)&0xffffffff
        H[1] = (H[1] + b)&0xffffffff
        H[2] = (H[2] + c)&0xffffffff
        H[3] = (H[3] + d)&0xffffffff
        H[4] = (H[4] + e)&0xffffffff

    string = f'{hex(H[0])[2:]}{hex(H[1])[2:]}{hex(H[2])[2:]}{hex(H[3])[2:]}{hex(H[4])[2:]}'
    return string

In [114]:
mac = "dca7119efa4033545d766af94f022809bd629c67"
key = "0"*16
m1 = "No one has completed Project #3 so give them all a 0."
m2 = "abahour2 grade = 100"
m_hex = "4E6F206F6E652068617320636F6D706C657465642050726F6A65637420233320736F2067697665207468656D20616C6C206120302E8000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000228616261686F757232206772616465203D20313030"
m = key + m1
m = Pad(m)
#print(m)
m_string = ""
for i in range(0,len(m),8):
    an_integer = int(m[i:i+8], 2)
    # if an_integer < 32 or an_integer > 127:
    #     m_string += "0"
    # else:
    m_string += chr(an_integer)

m_string += m2
#print(m_string)
#print(len(m_string))
print(SHA1_MOD(mac, m_string))

a54315fb69ecee68f6b94b691d34b9d9253c14c3
